In [7]:
import requests
import json
from pathlib import Path
import os
import sys
sys.path.insert(0, os.path.abspath(os.path.join('..', 'src')))

In [18]:
clientCrt = "../certificates/rw.crt"
clientKey = "../certificates/rw.key"

apiBaseUrl = "https://docucamrw.hesotech.eu/DocuCam/CameraOne/api/v1"
channelInfoUrl = apiBaseUrl + "/Data/ChannelInfo"
imageApiUrl = apiBaseUrl + "/Data/ImageAndMeasurements"  # measurements are averages during scan time (scan time 5-30min)

headers = {'content-type': 'application/json'}

In [27]:
requestData = {
    'SiteName': 'CameraTwo',
    'SceneName': 'SmallFieldSlide',
    'TimeRange': {
        'Minimum': '2023-01-01T00:00:00.000Z',
        'Maximum': '2023-02-01T00:00:00.000Z'
    },
    'Coordinates': [ { 'Layer': 0, 'Row': 1, 'Col': 0 }, { 'Layer': 0, 'Row': 1, 'Col': 1 } ]  

}

In [28]:
channelResponse = requests.get(channelInfoUrl, headers=headers, cert=(clientCrt, clientKey), allow_redirects=True)

if (channelResponse.status_code != 200):
    print('Something went wrong!')
    exit(-1)

In [29]:
response = requests.post(
    imageApiUrl,
    headers=headers,
    cert=(clientCrt, clientKey),
    allow_redirects=True,
    data=json.dumps(requestData)
)

if (response.status_code != 200):
    print('Something went wrong!')
    exit(-1)


In [30]:
channelInfoData = channelResponse.json()
data = response.json()
data[0]
# for record in data:
#     print(record['Timestamp'], ' -> ', record['Values'], ' -> ', record['ImageUrls'])

{'Timestamp': '2023-01-01T08:08:45.293837+00:00',
 'Values': [17.700000762939453,
  57,
  100000,
  718.2999877929688,
  5953,
  17.899999618530273,
  4.972222328186035,
  0,
  0,
  127,
  2],
 'ImageUrls': ['https://docucamrw.hesotech.eu/DocuCam/CameraOne/Image/1/0/1/0/2023-01-01T08-08Z',
  'https://docucamrw.hesotech.eu/DocuCam/CameraOne/Image/1/0/1/1/2023-01-01T08-08Z']}